In [1]:
import numpy as np
from qiskit import QuantumCircuit, Aer, assemble

In [1]:
class circuit:
    
    H = np.array([[1,1],[1,-1]]) / np.sqrt(2)
    S = np.array([[1,0],[0,0+1j]])
    
    def __init__(self, filename):
        self.Nqubits = 0
        self.Gates = []
        
        
        with open(filename, 'r') as reader:
            self.Nqubits = int(reader.readline().split()[0])
            
            for line in reader:
                if line  != '\n':
                    self.Gates.append([line.split()[0]] + [int(q) for q in line.split()[1:]])
        
        self.State = np.eye(2**self.Nqubits)
        self.SingleBitOps = [np.eye(2) for i in range(self.Nqubits)]
    
    def simulate(self):
        for gate in self.Gates:
            if gate[0] == 'H':
                self.SingleBitOps[gate[1]] = self.H @ self.SingleBitOps[gate[1]]
            elif gate[0] == 'S':
                self.SingleBitOps[gate[1]] = self.S @ self.SingleBitOps[gate[1]]
            elif gate[0] == 'CX':
                TempOp1 = self.SingleBitOps[-1]
                for i in range(len(self.SingleBitOps)-2, -1, -1):
                    TempOp1 = np.kron(TempOp1, self.SingleBitOps[i])

                self.SingleBitOps = [np.eye(2) for i in range(self.Nqubits)]
                self.SingleBitOps[gate[1]] = np.array([[1,0],[0,0]])
                
                TempOp2 = self.SingleBitOps[-1]
                for i in range(len(self.SingleBitOps)-2, -1, -1):
                    TempOp2 = np.kron(TempOp2, self.SingleBitOps[i])
                
                self.SingleBitOps[gate[1]] = np.array([[0,0],[0,1]])
                self.SingleBitOps[gate[2]] = np.array([[0,1],[1,0]])
                
                TempOp3 = self.SingleBitOps[-1]
                for i in range(len(self.SingleBitOps)-2, -1, -1):
                    TempOp3 = np.kron(TempOp3, self.SingleBitOps[i])
                
                
                self.State =(TempOp2 + TempOp3) @ TempOp1 @ self.State 
                
                self.SingleBitOps = [np.eye(2) for i in range(self.Nqubits)]
        
        TempOp = self.SingleBitOps[-1]
        for i in range(len(self.SingleBitOps)-2, -1, -1):
            TempOp = np.kron(TempOp, self.SingleBitOps[i])
        self.SingleBitOps = [np.eye(2) for i in range(self.Nqubits)]
        self.State = TempOp @ self.State
        
    
    def measure(self):
        InitState = np.zeros(2**self.Nqubits)
        InitState[0] = 1
        
        return self.State @ InitState


NameError: name 'np' is not defined

In [3]:
#C1 = circuit('Circuit1.txt')
C1 = circuit('Circuit1.txt')
C1.simulate()
print(C1.measure())

[0.5 0.  0.  0.5 0.5 0.  0.  0.5]


In [4]:
def test_result(circuit, res):
    qc = QuantumCircuit(circuit.Nqubits)
    for gate in circuit.Gates:
        if gate[0] == 'H':
            qc.h(gate[1])
        elif gate[0] == 'S':
            qc.s(gate[1])
        elif gate[0] == 'CX':
            qc.cx(gate[1], gate[2])
    
    # Let's get the result:
    svsim = Aer.get_backend('statevector_simulator')
    qobj = assemble(qc)
    result = svsim.run(qobj).result()

    final_state = result.get_statevector()
    
    if np.allclose(abs(final_state), abs(res)):
        return True
    else:
        return False
        

In [5]:
ntests = 20
gatetypes = ['H', 'S', 'CX']

for i in range(ntests):
    fname = 'TestCircuit' + str(i) + '.txt'
    with open(fname, 'w') as writer:
        qbits = np.random.randint(2,10)
        writer.write(str(qbits) + ' Qubits \n\n')
        ngates = np.random.randint(2,50)
        for j in range(ngates):
            g = np.random.randint(0,3)
            if gatetypes[g] == 'CX':
                c = np.random.randint(0, qbits)
                t = c
                while t == c:
                    t = np.random.randint(0, qbits)
                ss = str(gatetypes[g]) + ' ' + str(c) + ' ' + str(t) + '\n'
                writer.write(ss)
            else:
                t = np.random.randint(0, qbits)
                ss = str(gatetypes[g]) + ' ' + str(t) + '\n'
                writer.write(ss)
 


In [64]:
for i in range(ntests):
    fname = 'TestCircuit' + str(i) + '.txt'
    circ = circuit(fname)
    circ.simulate()
    res = circ.measure()
    if test_result(circ, res):
        print('Passed')
    else:
        print('Failed')

Passed
Passed
Passed
Passed
Passed
Passed
Passed
Passed
Passed
Passed
Passed
Passed
Passed
Passed
Passed
Passed
Passed
Passed
Passed
Passed
